In [1]:
from datasets import load_dataset
from dotenv import load_dotenv, find_dotenv
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
from sentence_transformers import SentenceTransformer

In [2]:
fw = load_dataset("HuggingFaceFW/fineweb", name= "sample-10BT", split= "train", streaming= True)

Resolving data files:   0%|          | 0/25868 [00:00<?, ?it/s]

In [3]:
fw

IterableDataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    num_shards: 15
})

In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
load_dotenv(find_dotenv(), override= True)

True

In [6]:
pc = Pinecone(api_key= os.environ.get("PINECONE_API_KEY"), enviroment = os.environ.get("PINECONE_ENV"))

In [7]:
pc.list_indexes()

[
    {
        "name": "text",
        "metric": "cosine",
        "host": "text-03idc8k.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "my-index-3",
        "metric": "dotproduct",
        "host": "my-index-3-03idc8k.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1000,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "my

In [9]:
pc.list_indexes()

[
    {
        "name": "text",
        "metric": "cosine",
        "host": "text-03idc8k.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "my-index-3",
        "metric": "dotproduct",
        "host": "my-index-3-03idc8k.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1000,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "my

In [10]:
index = pc.Index(name= "text")

In [11]:
# Define the number of items you want to process (subset size)
subset_size = 1500 # For example, take only 1500 items

# Iterate over the dataset and prepare data for upserting
vectors_to_upsert = []
for i, item in enumerate(fw):
    if i >= subset_size:
        break
        
    text = item['text']
    unique_id = str(item['id'])
    language = item['language']
    
    # Create an embedding for the text
    embedding = model.encode(text, show_progress_bar=False).tolist()

    # Prepare metadata
    metadata = {'language': language}

    # Append the tuple (id, embedding, metadata) to the list
    vectors_to_upsert.append((unique_id, embedding, metadata))

# Upsert data to Pinecone in batches
batch_size = 1000  # Adjust based on your environment and dataset size
for i in range(0, len(vectors_to_upsert), batch_size):
    batch = vectors_to_upsert[i:i + batch_size]
    index.upsert(vectors=batch)

print("Subset of data upserted to Pinecone index.")

Subset of data upserted to Pinecone index.
